In [3]:
# get environment variables
import os
from dotenv import load_dotenv, find_dotenv

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()

# load up the entries as environment variables
load_dotenv(dotenv_path)

import sys
sys.path.append(os.environ.get('srcdir'))

import features.rotate_vector_roms

In [6]:
import xarray as xr
tides_path = os.path.join(os.environ.get('intdir'),'waom10_tds.nc')
tides_path

'/home/ubuntu/bigStick/tidal_melting/data/preprocessing/interim/waom10_tds.nc'

In [9]:
tides = xr.open_dataset(tides_path)
tides.tide_Cangle.attrs

OrderedDict([('long_name', 'tidal current inclination angle'),
             ('units', 'degrees between semi-major axis and East'),
             ('field', 'tide_Cangle, scalar ')])

In [ ]:
# %load ../../src/features/rotate_vector_roms.py
from netCDF4 import Dataset
from numpy import *

# Given a 2D vector in x-y space on the ROMS grid (x component on the u-grid,
# y component on the v-grid), interpolate them both to the rho-grid and rotate
# the vector to lon-lat space.
# Input:
# u = x-component of vector on the ROMS u-grid
# v = y-component of vector on the ROMS v-grid
# angle = angle between the ROMS x-axis and east, at each point, in radians
# Output:
# u_lonlat, v_lonlat = components of the vector with respect to lon-lat space
def rotate_vector_roms2NE (u, v, angle):

    # Interpolate u to the rho-grid
    w_bdry_u = u[:,0]
    middle_u = 0.5*(u[:,0:-1] + u[:,1:])
    e_bdry_u = u[:,-1]
    u_rho = ma.concatenate((w_bdry_u[:,None], middle_u, e_bdry_u[:,None]), axis=1)
    # Interplate v to the rho-grid
    s_bdry_v = v[0,:]
    middle_v = 0.5*(v[0:-1,:] + v[1:,:])
    n_bdry_v = v[-1,:]
    v_rho = ma.concatenate((s_bdry_v[None,:], middle_v, n_bdry_v[None,:]), axis=0)

    # Rotate
    uv = (u_rho+1j*v_rho)*exp(1j*angle)
    u_lonlat = uv.real
    v_lonlat = uv.imag

    return u_lonlat, v_lonlat

def rotate_vector_NE2roms (u, v, angle):

    # Rotate
    uv = (u_rho+1j*v_rho)*exp(1j*-angle)
    u_lonlat = uv.real
    v_lonlat = uv.imag

    return u_roms, v_roms
    
